In [ ]:
import mlflow
from mlflow.entities import Run

from pasteur.kedro.ipython import *
register_kedro()

INFO     Kedro project Pasteur                                                                                           __init__.py:77
INFO     Defined global variable 'context', 'session', 'catalog' and 'pipelines'                                         __init__.py:78
INFO     Registered line magic 'run_viz'                                                                                 __init__.py:84


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pasteur.kedro.mlflow import get_run_name, get_parent_name, get_run_name, get_run

In [ ]:
parent = "tab_adult.privbayes.synth -h e1=[0.1,0.2,0.5] (git:73599410)"
runs = {
    "tab_adult.privbayes.synth e1=0.1 (git:73599410)": {"e1": 0.1},
    "tab_adult.privbayes.synth e1=0.2 (git:73599410)": {"e1": 0.2},
    "tab_adult.privbayes.synth e1=0.5 (git:73599410)": {"e1": 0.5},
}


In [ ]:
run = get_run(next(iter(runs.keys())), parent)
run



<Run: data=<RunData: metrics={}, params={'alg._name': 'privbayes',
 'alg.c': '1.5',
 'alg.e1': '0.3',
 'alg.e2': '0.7',
 'alg.ep': '0.1',
 'alg.random_init': 'False',
 'alg.rebalance_columns': 'True',
 'alg.theta': '4',
 'alg.u': '1.3',
 'alg.unbounded_dp': 'True',
 'e1': '0.1',
 'perf.multiprocess': 'True',
 'perf.synth.gpu': 'False',
 'random_state': '1234212',
 'ratios': "{'wrk': 0.8, 'ref': 0.2}",
 'view': 'tab_adult'}, tags={'extra_params': "{'e1': 0.1, '_mlflow_parent_name': "
                 "'tab_adult.privbayes.synth -h e1=[0.1,0.2,0.5] "
                 "(git:73599410)'}",
 'kedro_version': '0.18.3',
 'mlflow.parentRunId': '5bae4c399f97471ba48738b9035972ac',
 'mlflow.runName': 'tab_adult.privbayes.synth e1=0.1 (git:73599410)',
 'mlflow.source.git.commit': '73599410b723cbc3d0cb93f308770956d9e02f8d',
 'mlflow.source.name': '/mnt/ext/projects/pasteur/venv/bin/kedro',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'dev',
 'pasteur_id': 'tab_adult.privbayes.synth e1=0.1 (git:7

In [ ]:
import json
import pickle
import os

from pasteur.metrics.mlflow import ARTIFACT_DIR

def get_run_artifacts(run: Run):
    artifact_dir = mlflow.artifacts.download_artifacts(run_id=run.info.run_id, artifact_path=ARTIFACT_DIR)

    artifacts = {}
    # Load all artifacts by walking
    for root, _, files in os.walk(artifact_dir):
        if not files:
            continue
        
        # if dir is <p1>/<p2>/<p3> place artifacts in {p1: {p2: {p3: artifacts}}}
        sub_dict = artifacts
        for sub in root.replace(artifact_dir, "").split("/"):
            if not sub:
                continue
            tmp = sub_dict.get(sub, {})
            sub_dict[sub] = tmp
            sub_dict = tmp

        # load all files, support pickle and json for now
        for name in files:
            fn = os.path.join(root, name)
        
            with open(fn, "rb") as f:
                if fn.endswith(".json"):
                    art = json.load(f)
                elif fn.endswith(".pkl"):
                    art = pickle.load(f)

            no_ext = name[:name.rindex(".")]
            sub_dict[no_ext] = art

    return artifacts

artifacts = get_run_artifacts(run)
artifacts


{
    'meta': {'_tables': {'table': {'primary_key': 'id', 'metrics': TableMetrics(model=TableModelMetrics(expand_table=True, targets=[], sensitive=[])), '_columns': {'age': {'type': 'numerical', 'dtype': None, 'ref': None, 'metrics': MetricsMeta(x_log=False, y_log=False, bins=20, x_min=None, x_max=None, y_min=None, y_max=None), 'args': {'type': 'numerical'}}, 'workclass': {'type': 'categorical', 'dtype': None, 'ref': None, 'metrics': MetricsMeta(x_log=False, y_log=True, bins=20, x_min=None, x_max=None, y_min=None, y_max=None), 'args': {'type': 'categorical', 'metrics': {'y_log': True}}}, 'fnlwgt': {'type': 'numerical', 'dtype': None, 'ref': None, 'metrics': MetricsMeta(x_log=False, y_log=False, bins=20, x_min=None, x_max=None, y_min=None, y_max=None), 'args': {'type': 'numerical'}}, 'education': {'type': 'categorical', 'dtype': None, 'ref': None, 'metrics': MetricsMeta(x_log=False, y_log=False, bins=20, x_min=None, x_max=None, y_min=None, y_max=None), 'args': {'type': 'categorical'}},

In [ ]:
def get_artifacts(runs: dict[str, Run]):
    return {name: get_run_artifacts(get_run(name, parent)) for name in runs}

artifacts = get_artifacts(runs)
list(artifacts.keys())


[
    'tab_adult.privbayes.synth e1=0.1 (git:73599410)',
    'tab_adult.privbayes.synth e1=0.2 (git:73599410)',
    'tab_adult.privbayes.synth e1=0.5 (git:73599410)'
]


In [ ]:
assert len(artifacts)
ref_artifacts = next(iter(artifacts.values()))
visual = ref_artifacts["visual"]
meta = ref_artifacts["meta"]
meta.tables

['table']


In [ ]:
holder = visual["holder"]
data = {
    "wrk": visual["wrk"],
    "ref": visual["ref"]
}
syn_data = {name: art["visual"]["syn"] for name, art in artifacts.items()}
data.update(syn_data)

In [ ]:
from pasteur.metrics.visual import mlflow_log_hists
# mlflow_log_hists(holder, log_artifacts=False, **data)


[
    'wrk',
    'ref',
    'tab_adult.privbayes.synth e1=0.1 (git:73599410)',
    'tab_adult.privbayes.synth e1=0.2 (git:73599410)',
    'tab_adult.privbayes.synth e1=0.5 (git:73599410)'
]


In [ ]:
from typing import Any

def gen_run_names(run_params: dict[str, dict[str, Any]]):
    """Generates a run name based on parameters that are short for use in graphs.

    All names have the same length.
    
    Parameters that start with `_`, get priority and only have their value printed.
    Ex. `{"_alg": "privbayes", "e1": "abc"}` becomes `privbayes e_1: abc`.
    
    Parameters composed of letters and then numbers have their number become an indicator:
    `e1` becomes `e_1`, where `_` indicates subscript. TODO
    
    Parameters with boolean are only printed when true. """

    ref_run = next(iter(run_params.values()))
    value_params = {k for k in ref_run if k.startswith("_")}
    bool_params = {k for k,v in ref_run.items() if isinstance(v, bool)}

    str_params = {name: [] for name in run_params}
    for param in ref_run:
        # Calculate str length for str_params
        length = max(map(lambda x: len(str(x)), [run[param] for run in run_params.values()]))
        
        for name in run_params:
            if param in bool_params:
                s = param if run_params[name][param] else (" " * len(param))
            else:
                val_str = str(run_params[name][param])
                buffer = " " * (length - len(val_str))

                if param in value_params:
                    s = f"{buffer}{val_str}"
                else:
                    s = f"{param}={buffer}{val_str}"
        
            str_params[name].append(s)
        
    return {name: " ".join(params) for name, params in str_params.items()}


runs_tst = {
    'tab_adult.privbayes.synth e1=0.1 (git:73599410)': {"_alg": "privbayes", 'e1': 0.1, "rebalance_columns": True},
    'tab_adult.privbayes.synth e1=0.2 (git:73599410)': {"_alg": "aim", 'e1': 0.2, "rebalance_columns": True},
    'tab_adult.privbayes.synth e1=0.5 (git:73599410)': {"_alg": "privmrf", 'e1': 0.5, "rebalance_columns": True}
}
gen_run_names(runs_tst)



{
    'tab_adult.privbayes.synth e1=0.1 (git:73599410)': 'privbayes e1=0.1 rebalance_columns',
    'tab_adult.privbayes.synth e1=0.2 (git:73599410)': '      aim e1=0.2 rebalance_columns',
    'tab_adult.privbayes.synth e1=0.5 (git:73599410)': '  privmrf e1=0.5 rebalance_columns'
}
